In [35]:
import tensorflow as tf
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.3333)
#sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

import os
os.environ["CUDA_DEVICE_ORDER"]='PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"]='2'

import keras
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dense, Lambda, Flatten
from keras.optimizers import Adam, sgd
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers.embeddings import Embedding
from keras.regularizers import l1, l2

import pandas as pd
import numpy as np

#if 'session' in locals() and session is not None:
#    print('Close interactive session')
#    session.close()

path = "D:\\jtownend\\fast.ai\\data\\movielens\\"

In [28]:
ratings = pd.read_csv(path+'ratings.csv')
movie_names = pd.read_csv(path+'movies.csv').set_index('movieId')['title'].to_dict()

users = ratings.userId.unique()
movies = ratings.movieId.unique()
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()

#replace the user and movie IDs with contiguous integers
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])

#divide randomly into train and validation sets
np.random.seed = 42
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]

#set number of latent factors
n_factors = 50

## Dot product

In [58]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users, n_factors, input_length=1, W_regularizer=l2(1e-4))(user_in)
movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
m = Embedding(n_movies, n_factors, input_length=1, W_regularizer=l2(1e-4))(movie_in)

x = merge([u, m], mode='dot')
x = Flatten()(x)
model = Model([user_in, movie_in], x)
model.compile(loss='mse', optimizer=Adam(0.01), metrics=['accuracy'])

In [60]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=4, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80084 samples, validate on 19920 samples
Epoch 1/4
80084/80084 [==============================] - 6s - loss: 2.9582 - acc: 0.2762 - val_loss: 3.5573 - val_acc: 0.2539
Epoch 2/4
80084/80084 [==============================] - 6s - loss: 2.8840 - acc: 0.2828 - val_loss: 3.4992 - val_acc: 0.2536
Epoch 3/4
80084/80084 [==============================] - 6s - loss: 2.8227 - acc: 0.2906 - val_loss: 3.4855 - val_acc: 0.2560
Epoch 4/4
80084/80084 [==============================] - 7s - loss: 2.7775 - acc: 0.2928 - val_loss: 3.4578 - val_acc: 0.2556


In [61]:
model.optimizer.lr=0.001

In [62]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=3, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80084 samples, validate on 19920 samples
Epoch 1/3
80084/80084 [==============================] - 6s - loss: 2.7462 - acc: 0.2954 - val_loss: 3.4222 - val_acc: 0.2560
Epoch 2/3
80084/80084 [==============================] - 6s - loss: 2.7214 - acc: 0.2973 - val_loss: 3.4220 - val_acc: 0.2559
Epoch 3/3
80084/80084 [==============================] - 6s - loss: 2.7115 - acc: 0.2976 - val_loss: 3.4147 - val_acc: 0.2596


## Introduce bias to improve result